In [1]:
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [2]:
batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'


In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
for line in lines[: min(num_samples, len(lines))]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [6]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [7]:
max_encoder_seq_length = max([len(line) for line in input_texts]) + 1
max_decoder_seq_length = max([len(line) for line in target_texts]) + 1

In [8]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 17
Max sequence length for outputs: 60


In [9]:
input_token_index = dict([(c, i) for i, c in enumerate(input_characters) ])
target_token_index = dict([(c, i) for i, c in enumerate(target_characters) ])

In [10]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')


In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, c in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[c]] = 1.
        
    encoder_input_data[i, t+1, input_token_index[' ']] = 1.
    
    #  h e l l o \s 
    # \t h e l l o
    for t, c in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[c]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t-1, target_token_index[c]] = 1.
            
    decoder_input_data[i, t+1, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    

In [12]:
# MODEL

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 70)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 334848      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  358400      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [13]:
# save model
model.save('s2s.h5')

In [14]:
# INFERENCE MODELS

# ENCODER
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()


# DECODER
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, next_state_h, next_state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
next_state = [next_state_h, next_state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + next_state)
decoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 70)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 334848    
Total params: 334,848
Trainable params: 334,848
Non-trainable params: 0
_________________________________________________________________
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
___

In [15]:
input_index_token = {i: c for c, i in input_token_index.items()}
target_index_token = {i: c for c, i in target_token_index.items()}

def generate_seq(input_seq):
    states = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    stop = False
    decoded_sentence = ''
    
    while not stop:        
        o, h, c = decoder_model.predict([target_seq] + states)
        
        sampled_token_index = np.argmax(o[0, -1, :])
        sampled_char = target_index_token[sampled_token_index]
        decoded_sentence += sampled_char
        
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, target_token_index[sampled_char]] = 1.        
        states = [h, c]
        
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop = True
            
    return decoded_sentence


for i in range(0, 100):
    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = generate_seq(input_seq)
    print('-')
    print('Input ', input_texts[i])
    print('Output ', decoded_sentence)
    
        
        
        

-
Input  Go.
Output  Je s  s                                                      
-
Input  Hi.
Output  Je s     s                                                   
-
Input  Hi.
Output  Je s     s                                                   
-
Input  Run!
Output  Je s    s                                                    
-
Input  Run!
Output  Je s    s                                                    
-
Input  Who?
Output  Je  s    s                                                   
-
Input  Wow!
Output  Je  s                                                        
-
Input  Fire!
Output  Je s  s                                                      
-
Input  Help!
Output  Je s     s                                                   
-
Input  Jump.
Output  Je s     s                                                   
-
Input  Stop!
Output  Je s   s                                                     
-
Input  Stop!
Output  Je s   s                                            

-
Input  Call me.
Output  Je s  s                                                      
-
Input  Call us.
Output  Je s  s                                                      
-
Input  Call us.
Output  Je s  s                                                      
-
Input  Come in.
Output  Je s    s                                                    
-
Input  Come in.
Output  Je s    s                                                    
